In [1]:
import numpy as np
from IPython.display import display, Latex

In [2]:
def tildeAlpha(g, i, n):
    alpha_mat = np.identity(3*g-1)
    alpha_mat[i-1, g-1+i] = n

    return alpha_mat

In [3]:
tildeAlpha(2,1,5)

array([[1., 0., 5., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

In [4]:
tildeAlpha(2,2,5)

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 5., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

In [302]:
def tildeBeta(g,i,n):
    beta_mat = np.identity(3*g-1)
    if g == 1:
        beta_mat[g+i-1, i-1] = n
    elif g > 1:
        if i == 1:
            beta_mat[g-1+i, i-1] = n
            beta_mat[g-1+i, g-1+i] = 1
            beta_mat[g-1+i, 2*g+i-1] = n

        elif i == g:
            beta_mat[g-1+i, i-1] = n
            beta_mat[g-1+i, g-1+i] = 1
            beta_mat[g-1+i, 2*g-2+i] = n

        else:
            beta_mat[g-1+i, i-1] = n
            beta_mat[g-1+i, g-1+i] = 1
            beta_mat[g-1+i, 2*g-1+i] = n
            beta_mat[g-1+i, 2*g+i] = n

    else:
        raise('The value g represents the of the surface and is assumed to be positive!')

    return beta_mat

In [159]:
tildeBeta(1,1,1)

array([[1., 0.],
       [1., 1.]])

In [317]:
tildeBeta(2,1,5)

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [5., 0., 1., 0., 5.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

In [318]:
tildeBeta(2,2,5)

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 5., 0., 1., 5.],
       [0., 0., 0., 0., 1.]])

In [166]:
def tildeGamma(g,i,n):
    gamma_mat = np.identity(3*g-1)
    gamma_mat[2*g+i-1, g+i-1] = n 
    gamma_mat[2*g+i-1, g+i] = n
    
    return gamma_mat

In [167]:
tildeGamma(2,1,5)

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 5., 5., 1.]])

In [168]:
display(Latex('$x_1^2$'))

<IPython.core.display.Latex object>

In [11]:
a = 'A'

In [309]:
def getPresentationMatrix(g, Sigma, fundamental_group = True):
    A = np.zeros((g,g))
    images_alpha = []
    if fundamental_group:
        fg_string = ''
        relators = '|'

    #iterate over the homeomorphism in order of the word actions
    SigmaList  = Sigma.split('.')
    SigmaList.reverse()
    
    #iterate over each alpha_i
    for i in range(g):
            
        #v_i will represent the image Sigma(alpha_i) at all steps
        v_i = np.zeros(3*g-1)
        v_i[i] = 1

        #add generator
        if fundamental_group:
            fg_string += 'x_' + str(i+1)
            betas = v_i[g:2*g]
            
        for j in SigmaList:
            #get the type of curve which the Dehn twist is applied
            letter = j[0]

            #get the index of the Dehn twist
            after_subindex = j.split('_')[-1]
            if after_subindex[0] == '{':
                index = int(after_subindex.split('{')[1].split('}')[0])
            else:
                index = int(after_subindex[0])

            #get the exponential of the Dehn twist
            after_exponential = j.split('^')
            if len(after_exponential) != 1:
                after_exponential = after_exponential[-1]
                if after_exponential[0] == '{':
                    exponential = int(after_exponential.split('{')[1].split('}')[0])
                else:
                    exponential = int(after_exponential[0])
            else:
                exponential = 1
        
            if letter == 'a':
                v_i = tildeAlpha(g, index, exponential) @ v_i
            elif letter == 'b':
                v_i = tildeBeta(g, index, exponential) @ v_i
            elif letter == 'c':
                v_i = tildeGamma(g, index, exponential) @ v_i
            else:
                 raise Exception("Incorrect surface diffeomorphism!")
           
            #if any of the betas changed on this step, we add the corresponding relator to 
            if fundamental_group:
                for count, beta in enumerate(v_i[g:2*g].tolist()): 
                    if beta != betas[count]:
                        relators += 'x_{' + str(count+1) + '}^{' + str(int(beta-betas[count])) + '}'
                betas = v_i[g:2*g]
                    
                
        A[i] = v_i[g:2*g]
        images_alpha.append(list(v_i))

        if fundamental_group:
            if i != g-1:
                fg_string += ', '
                relators += ', '
    
    if fundamental_group:
        display(Latex( '$' + fg_string + relators + '$'))

    return A, images_alpha

In [310]:
getPresentationMatrix(1, 'a_1^{-1}.b_1.a_1')

<IPython.core.display.Latex object>

(array([[1.]]), [[0.0, 1.0]])

In [311]:
getPresentationMatrix(1, 'a_1^{-1}.b_1^{10}.a_1')

<IPython.core.display.Latex object>

(array([[10.]]), [[-9.0, 10.0]])

In [312]:
getPresentationMatrix(2, 'a_1^{-1}.b_1.a_1')

<IPython.core.display.Latex object>

(array([[1., 0.],
        [0., 0.]]),
 [[0.0, 0.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0, 0.0]])

In [313]:
getPresentationMatrix(2, 'b_2.c_1.b_1')

<IPython.core.display.Latex object>

(array([[1., 1.],
        [0., 1.]]),
 [[1.0, 0.0, 1.0, 1.0, 1.0], [0.0, 1.0, 0.0, 1.0, 0.0]])

In [314]:
getPresentationMatrix(2, 'a_1^{-1}.b_1^6.a_1.b_1^4.c_1^4.b_2^4.a_2^{-1}.b_2.a_2')

<IPython.core.display.Latex object>

(array([[ 34.,   0.],
        [136.,   1.]]),
 [[-29.0, 0.0, 34.0, 0.0, 0.0], [-120.0, 0.0, 136.0, 1.0, 4.0]])

In [315]:
getPresentationMatrix(2, 'b_1^4.a_1^{-1}.b_1.a_1')

<IPython.core.display.Latex object>

(array([[1., 0.],
        [0., 0.]]),
 [[0.0, 0.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0, 0.0]])

In [316]:
tildeBeta(2, 1, 4)

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [4., 0., 1., 0., 4.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])